In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True


In [2]:
#print(df['Page content'][2])

#content = procd.parse_html(df['Page content'][2])


In [2]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
"""
parse_train = np.zeros(df.values.shape, dtype=object)
parse_test = np.zeros(df_test.values.shape, dtype=object)

for i in range(df.values.shape[0]):
    data = df.values[i]
    content = procd.parse_html(data[2])
    parse_train[i, 0:2] = data[0:2]
    parse_train[i, 2] = content

parse_df = pd.DataFrame(data=parse_train, columns=df.columns)
parse_df.to_csv('train_parsed.csv', index=False)
    
for i in range(df_test.values.shape[0]):
    data = df_test.values[i]
    content = procd.parse_html(data[1])
    parse_test[i, 0] = data[0]
    parse_test[i, 1] = content

parse_df_test = pd.DataFrame(data=parse_test, columns=df_test.columns)
parse_df_test.to_csv('test_parsed.csv', index=False)
"""
"""

for i in df.values:
    topics = procd.parse_topic(i[2])
    for t in topics:
        dic[t] = dic.get(t, 0) + 1
        if i[1] == 1: pop[t] = pop.get(t, 0) + 1
print(dic)
print(pop)
"""
"""
a = 0
train_times = np.zeros((df.values.shape[0], 8), dtype=float)
test_times = np.zeros((df_test.values.shape[0], 8), dtype=float)

for i in df.values:
    text = BeautifulSoup(i[2], 'html.parser')
    weekday, train_times[7] = procd.parse_date(text.find('time')['datetime'])
    train_times[weekday] = 1

for i in df_test.values:
  
    text = BeautifulSoup(i[1], 'html.parser')
    try:
        weekday, test_times[7] = procd.parse_date(text.find('time')['datetime'])
    except:
        weekday = 0
        test_times[7] = 0
    test_times[weekday] = 1
cd = pd.DataFrame(data=train_times)
cd.to_csv('train_times.csv',index=False)
td = pd.DataFrame(data=test_times)
td.to_csv('test_times.csv', index=False)
"""
"""
ch = np.zeros((df.values.shape[0], 33))

for i in df.values:
    key = procd.parse_channel(i[2])
    ch[a, procd.dic[key]] = 1
    a += 1

cd = pd.DataFrame(data=ch)
cd.to_csv('train_channel.csv',index=False)

df_test = pd.read_csv('test.csv')
cn = np.zeros((df_test.values.shape[0], 33))
a=0
for i in df_test.values:
    key = procd.parse_channel(i[2])
    if key in dic:
        cn[a, dic[key]] = 1
        a += 1
td = pd.DataFrame(data=cn)
td.to_csv('test_channel.csv', index=False)
"""

"\nch = np.zeros((df.values.shape[0], 33))\n\nfor i in df.values:\n    key = procd.parse_channel(i[2])\n    ch[a, procd.dic[key]] = 1\n    a += 1\n\ncd = pd.DataFrame(data=ch)\ncd.to_csv('train_channel.csv',index=False)\n\ndf_test = pd.read_csv('test.csv')\ncn = np.zeros((df_test.values.shape[0], 33))\na=0\nfor i in df_test.values:\n    key = procd.parse_channel(i[2])\n    if key in dic:\n        cn[a, dic[key]] = 1\n        a += 1\ntd = pd.DataFrame(data=cn)\ntd.to_csv('test_channel.csv', index=False)\n"

In [7]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [36]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
p_data = df[df['Popularity'] > 0] 
n_data = df[df['Popularity'] < 0]

tokenizer_stem_nostop('amazon microsoft women hahahaha')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [14]:
import itertools, operator

def extract_term(top, tfidf):
    feat = tfidf.get_feature_names()
    idf = tfidf.idf_
    idx = idf.argsort()
    sorted_score = []
    
    for i in idx: sorted_score.append(idf[i])
    sorted_smallest = list(zip(feat, sorted_score))
    sorted_smallest = sorted(sorted_smallest, key=operator.itemgetter(1))
    sorted_highest = sorted_smallest[::-1]
    term = []
    print('[vocabularies with smallest idf scores]')
    for i in range(top):
        #print('%s : %f' %(sorted_smallest[i][0], sorted_smallest[i][1]))
        term.append(sorted_smallest[i][0])
        
    print('[vocabularies with height idf scores]')
    for i in range(top):
        #print('%s : %f' %(sorted_highest[i][0], sorted_highest[i][1]))
        term.append(sorted_highest[i][0])
    return term

In [15]:
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import _pickle as pkl

stop = stopwords.words('english')

p = p_data['Page content']
n = n_data['Page content']

if not_fit:
    tfidf = TfidfVectorizer(ngram_range=(1,1),
                        #preprocessor=procd.parse_html,
                        tokenizer=tokenizer_stem_nostop,
                        max_features=2**10,
                        max_df=0.6,
                        min_df=0.05,
                        sublinear_tf=True
                        )
    not_fit = False
    tfidf_p = tfidf.fit(p)
    pkl.dump(tfidf_p, open('tfidf_p.pkl', 'wb'))
    tfidf_n = tfidf.fit(n)
    pkl.dump(tfidf_n, open('tfidf_n.pkl', 'wb'))

tfidf_p = pkl.load(open('tfidf_p.pkl', 'rb'))
tfidf_n = pkl.load(open('tfidf_n.pkl', 'rb'))

t_p = extract_term(40, tfidf_p)
t_n = extract_term(40, tfidf_n)
   
extra_stop = []
for t in t_p:
    if t in t_n: extra_stop.append(t)
stop += extra_stop

print(stop)

[vocabularies with smallest idf scores]
[vocabularies with height idf scores]
[vocabularies with smallest idf scores]
[vocabularies with height idf scores]
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'm

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time
t0 = time()
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        #preprocessor=procd.parse_html,
                        tokenizer=tokenizer_stem_nostop,
                        max_features=2**16,
                        max_df=0.5,
                        min_df=0.05,
                        sublinear_tf=True
                        )
X_train = tfidf.fit_transform(df['Page content'])
duration = time() - t0

print('Finishing fitting TFIDF : %f s' %(duration))
print('dimension: %d' %(X_train.shape[1]))
#sc_x = MaxAbsScaler()
#X_train = sc_x.fit_transform(X_train)
y_train = df['Popularity']

Finishing fitting TFIDF : 180.395193 s
dimension: 711


In [33]:
from sklearn.feature_extraction.text import HashingVectorizer

t0 = time()
hashvec = HashingVectorizer(ngram_range=(1,2),
                           tokenizer=tokenizer_stem_nostop,
                           n_features=2**10)
X_train = hashvec.transform(df['Page content'])
duration = time() - t0
print('Finishing fitting HASH : %f s' %(duration))
print('dimension: %d' %(X_train.shape[1]))

Finishing fitting HASH : 151.906644 s
dimension: 1024


In [5]:
from scipy.sparse import hstack

train_channel = pd.read_csv('train_channel.csv').values
train_time = pd.read_csv('train_times.csv').values
X_train = hstack([X_train, train_channel, train_time])
#sc_x = StandardScaler(with_mean=False)
#X_train = sc_x.fit_transform(X_train)
print(X_train.shape)

(27643, 8233)


In [24]:
def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [38]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

batch_size = 2000

classes = np.array([-1, 1])
iters = int((27643+batch_size-1)/(batch_size))
Cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0]

best_score = 0
candidates = []
"""
t0 = time()
clf = SVC(max_iter=15000, random_state=0)
clf.fit(X_train, df['Popularity'])
candidates.append({'clf':clf, 'C':C, 'score':score})
score = clf.score(X_train, df['Popularity'])
print(score)
candidates.append({'clf':clf, 'C':0.001, 'score':score})
print('training duration : %f' %(time()-t0))
"""

#selector = SelectFromModel(estimator=LinearSVC(dual=False, tol=1e-3))
#X_train_sel = selector.fit_transform(X_train, y_train)
X_train_sel = X_train
#poly = PolynomialFeatures(degree=3)
#X_train_poly = poly.fit_transform(X_train_sel.toarray())
X_train_poly = X_train_sel
clf = MultinomialNB(alpha=0.01)
clf.fit(abs(X_train_sel), y_train)
pred = clf.predict(X_train_sel)

print("MNB score : %f" %(metrics.accuracy_score(y_train, pred)))
candidates.append({'clf':clf, 'C':0, 'score':score})

SGD = SGDClassifier(alpha=0.0001, loss='hinge', penalty='l2')
SGD.fit(X_train_sel, y_train)
pred = SGD.predict(X_train_sel)

print("SGD score : %f" %(metrics.accuracy_score(y_train, pred)))
candidates.append({'clf':SGD, 'C':1, 'score':score})


ELA = SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
ELA.fit(X_train_sel, y_train)
pred = ELA.predict(X_train_sel)

print("SGD score : %f" %(metrics.accuracy_score(y_train, pred)))
candidates.append({'clf':ELA, 'C':2, 'score':score})

"""
for C in Cs:
    clf = Pipeline([
            ('feature_selection', SelectFromModel(estimator=LinearSVC(penalty='l1', dual=False, tol=1e-3))),
            ('classification', LinearSVC(C=C))
        ])
    #clf = LinearSVC(C=C, dual=False, tol=0.00001, max_iter=2000)
    clf.fit(X_train_sel, y_train)
    pred = clf.predict(X_train_sel)

    score = metrics.accuracy_score(y_train, pred)
    print("C(%f):%f"%(C,score))
    candidates.append({'clf':clf, 'C':C, 'score':score})
"""
"""
for C in Cs:
    alpha = C / 1000.0
    clf = SGDClassifier(loss='hinge', alpha=alpha)
    #stream = get_stream(path='train.csv', size=batch_size)
    for i in range(iters):
        #batch = next(stream)
        #X_train, y_train = batch['Page content'], batch['Popularity']
        start = (i)*batch_size
        end = start + batch_size
        if end >= X_train.shape[0] : end = X_train.shape[0]
        
        batch_X = X_train[start:end]
        batch_y = y_train[start:end]
        if X_train is None:
            break
        #X_train = sc_x.transform(tfidf.transform(X_train))
        clf.partial_fit(batch_X, batch_y, classes=classes)
    
        score = clf.score(batch_X, batch_y)
        print('[{}/{}] {}'.format((i+1)*(batch_size), 27643, score))
    
    if score > best_score:
        print('bestscore(C=%f): %f' %(C, score))
        best_score = score
        candidates.append({'clf' : clf, 'C' : C, 'score' : score})
"""

MNB score : 0.506638
SGD score : 0.589914
SGD score : 0.589625


"\nfor C in Cs:\n    alpha = C / 1000.0\n    clf = SGDClassifier(loss='hinge', alpha=alpha)\n    #stream = get_stream(path='train.csv', size=batch_size)\n    for i in range(iters):\n        #batch = next(stream)\n        #X_train, y_train = batch['Page content'], batch['Popularity']\n        start = (i)*batch_size\n        end = start + batch_size\n        if end >= X_train.shape[0] : end = X_train.shape[0]\n        \n        batch_X = X_train[start:end]\n        batch_y = y_train[start:end]\n        if X_train is None:\n            break\n        #X_train = sc_x.transform(tfidf.transform(X_train))\n        clf.partial_fit(batch_X, batch_y, classes=classes)\n    \n        score = clf.score(batch_X, batch_y)\n        print('[{}/{}] {}'.format((i+1)*(batch_size), 27643, score))\n    \n    if score > best_score:\n        print('bestscore(C=%f): %f' %(C, score))\n        best_score = score\n        candidates.append({'clf' : clf, 'C' : C, 'score' : score})\n"

In [7]:
print(selector.get_support())
c = 0
for i in selector.get_support():
    if i == True: c += 1
print(c)

NameError: name 'selector' is not defined

In [28]:
t0 = time()
X_test = tfidf.transform(df_test['Page content'])
duration = time() - t0

print('Finishing fitting test TFIDF : %f s' %(duration))
#X_test = sc_x.transform(X_test)

Finishing fitting test TFIDF : 63.737878 s


In [39]:
t0 = time()
X_test = hashvec.transform(df_test['Page content'])
duration = time() - t0
print('Finishing fitting test Hashvec : %f s' %(duration))


Finishing fitting test Hashvec : 63.491124 s


In [8]:
test_channel = pd.read_csv('test_channel.csv').values
test_time = pd.read_csv('test_times.csv').values
X_test_con = hstack([X_test, test_channel, test_time])

#X_test_std = sc_x.fit_transform(X_test_con)

In [41]:
X_test_con = X_test

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test%f.csv' % (cand['C']), index=False)